# Optimización de Pesos con Umbral (Modelo Estricto V2)

Este notebook ajusta la optimización con la restricción adicional de que el quiz valga máximo la mitad que la tarea.

## Restricciones Finales Definidas
1.  **Simetría Estricta**: $Q_1=Q_2=Q_3$ y $T_1=T_2=T_3$.
2.  **Carga de Tareas**: Máximo 20% por módulo ($T_i \le 0.20$).
3.  **Examen Final**: Máximo 35% del total ($Final \le 0.35$).
4.  **Carga Mínima de Quices**: $\sum Q_i \ge 0.10$.
5.  **Proporción Quiz/Tarea**: $Q_i \le 0.5 \cdot T_i$ (NUEVA).

In [65]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

np.random.seed(42)
pd.set_option('display.float_format', '{:.2f}'.format)

In [66]:
# 1. GENERACIÓN DE DATOS
def generar_distribucion_bimodal(n, low_center=1.5, high_center=4.5, ratio=0.5, sigma=0.6):
    n_low = int(n * ratio)
    n_high = n - n_low
    vals = np.concatenate([
        np.random.normal(low_center, sigma, n_low),
        np.random.normal(high_center, sigma, n_high)
    ])
    return np.clip(vals, 0.0, 5.0)

N = 2000
m1_bases = generar_distribucion_bimodal(N, ratio=0.3)
m2_bases = generar_distribucion_bimodal(N, ratio=0.4)
m3_bases = generar_distribucion_bimodal(N, ratio=0.3)

m1_q = np.clip(m1_bases + np.random.normal(0, 0.5, N), 0, 5)
m1_t = np.clip(m1_bases + np.random.normal(0.2, 0.5, N), 0, 5)
m2_q = np.clip(m2_bases + np.random.normal(0, 0.5, N), 0, 5)
m2_t = np.clip(m2_bases + np.random.normal(0.2, 0.5, N), 0, 5)
m3_q = np.clip(m3_bases + np.random.normal(0, 0.5, N), 0, 5)
m3_t = np.clip(m3_bases + np.random.normal(0.2, 0.5, N), 0, 5)

avg_bases = (m1_bases + m2_bases + m3_bases) / 3.0
final_exam = np.clip(avg_bases - 0.5 + np.random.normal(0, 0.8, N), 0, 5)

df_notas = pd.DataFrame({
    'M1_Q': m1_q, 'M1_T': m1_t,
    'M2_Q': m2_q, 'M2_T': m2_t,
    'M3_Q': m3_q, 'M3_T': m3_t,
    'Examen_Final': final_exam
}).round(2)

In [67]:

def get_oracle_decisions_vectorized(weights, df):
    # Weights interpretation depends on length
    # If 3: [w_q, w_t, w_f] (Strict Notebook) -> Expand to 7
    # If 7: [m1q, m1t, m2q, m2t, m3q, m3t, final] (Standard Notebook)
    import numpy as np

    if len(weights) == 3:
        w_q, w_t, w_f = weights
        # Strict symmetry assumption
        weights_7 = np.array([w_q, w_t, w_q, w_t, w_q, w_t, w_f])
    else:
        weights_7 = weights

    # Normalize weights for module calculation (Dynamic)
    # We use the relative proportion of Q vs T in the current weights vector
    # to calculate the module grade.
    
    # Extract columns
    m1_q = df['M1_Q'].values
    m1_t = df['M1_T'].values
    m2_q = df['M2_Q'].values
    m2_t = df['M2_T'].values
    m3_q = df['M3_Q'].values
    m3_t = df['M3_T'].values
    final = df['Examen_Final'].values
    
    # Calculate Module Grades (Dynamic)
    # M1
    w1_tot = weights_7[0] + weights_7[1] + 1e-9
    n_m1 = (m1_q * weights_7[0] + m1_t * weights_7[1]) / w1_tot
    
    # M2
    w2_tot = weights_7[2] + weights_7[3] + 1e-9
    n_m2 = (m2_q * weights_7[2] + m2_t * weights_7[3]) / w2_tot
    
    # M3
    w3_tot = weights_7[4] + weights_7[5] + 1e-9
    n_m3 = (m3_q * weights_7[4] + m3_t * weights_7[5]) / w3_tot
    
    avg_modulos = (n_m1 + n_m2 + n_m3) / 3.0
    
    # Decision Logic (Vectorized)
    decisions = np.ones(len(df), dtype=int)
    
    # 1. MUERTE SÚBITA
    decisions[final < 2.5] = 0
    
    # 2. DEBILIDAD
    decisions[(final < 3.0) & (avg_modulos < 3.0)] = 0
    
    # 3. FILTRO M2 (Legacy strict rule)
    decisions[n_m2 < 2.5] = 0
    
    # 4. PROMEDIO TOTAL SIMPLE < 3.0 (Legacy rule check)
    # Note: Optimization notebooks often used simple average as baseline constraint
    # We'll calculate simple average of all raw components
    avg_total = (m1_q + m1_t + m2_q + m2_t + m3_q + m3_t + final) / 7.0
    # Actually, simpler: just check average of modules + final
    # decisions[(avg_modulos * 3 + final)/4 < 3.0] = 0
    # Let's stick to the notebook's specific legacy logic if it had one, or a general reasonable one.
    # The 'notas' notebook had: np.mean([all_cols]) < 3.0. Let's replicate roughly.
    all_raw_cols = np.stack([m1_q, m1_t, m2_q, m2_t, m3_q, m3_t, final], axis=1)
    decisions[np.mean(all_raw_cols, axis=1) < 3.0] = 0
    
    # REDEMTPION (Overrides previous fails)
    # If 2.5 <= Final < 3.0 BUT all modules > 3.5
    mask_redemption = (final >= 2.5) & (final < 3.0) & (n_m1 > 3.5) & (n_m2 > 3.5) & (n_m3 > 3.5)
    decisions[mask_redemption] = 1
    
    return decisions


In [68]:
# 3. LÓGICA DE UMBRAL (Estricta: 3 Variables)
def calcular_nota_strict(weights, df, U=2):
    w_q, w_t, w_f = weights
    w_mod = w_q + w_t
    eps = 1e-9
    
    n_m1 = (df['M1_Q']*w_q + df['M1_T']*w_t) / (w_mod + eps)
    n_m2 = (df['M2_Q']*w_q + df['M2_T']*w_t) / (w_mod + eps)
    n_m3 = (df['M3_Q']*w_q + df['M3_T']*w_t) / (w_mod + eps)
    n_fin = df['Examen_Final']
    
    comps = np.stack([n_m1, n_m2, n_m3, n_fin], axis=1)
    
    ratios = np.clip(comps / U, 0, 1)
    U_factors = np.zeros_like(comps)
    for i in range(4):
        others = [j for j in range(4) if j != i]
        U_factors[:, i] = np.prod(ratios[:, others], axis=1)
        
    global_weights = np.array([w_mod, w_mod, w_mod, w_f])
    
    # Calculate Standard Threshold Grade
    preds_strict = np.sum(comps * global_weights * U_factors, axis=1)
    
    # Calculate Raw Grade (Base)
    preds_raw = np.sum(comps * global_weights, axis=1)
    
    # STRICT FAILURE RULE:
    # If Raw Grade < 3.0, the grade is strictly set to 80% of Raw Grade.
    # We do NOT use the threshold calculation for these cases.
    fail_mask = preds_raw < 3.0
    fail_val = 0.8 * preds_raw
    
    return np.where(fail_mask, fail_val, preds_strict)

def funcion_objetivo(weights, df, y_ignored=None):
    # 1. Generar el "Ground Truth" Dinámico
    # El oraculo se recalcula con los pesos actuales
    y_true_dynamic = get_oracle_decisions_vectorized(weights, df)
    
    # 2. Calcular Notas Propuestas
    pred = calcular_nota_strict(weights, df)
    
    pass_th = 2.95
    err_pass = np.maximum(0, pass_th - pred[y_true_dynamic==1])**2
    err_fail = np.maximum(0, pred[y_true_dynamic==0] - pass_th)**2
    
    return np.sum(err_pass) + np.sum(err_fail)

cons = [
    {'type': 'eq', 'fun': lambda w: 3*w[0] + 3*w[1] + w[2] - 1},      # Suma = 1
    {'type': 'ineq', 'fun': lambda w: 0.20 - w[1]},                   # Tarea <= 20%
    {'type': 'ineq', 'fun': lambda w: 0.35 - w[2]},                   # Final <= 35%
    {'type': 'ineq', 'fun': lambda w: 3*w[0] - 0.10},                 # Total Quices >= 10%
    {'type': 'ineq', 'fun': lambda w: 0.5*w[1] - w[0]}                # Quiz <= 0.5 * Tarea
]
bnds = [(0.01, 1.0)]*3

print("Optimizando (Dynamic Oracle)...")
# Note: args changed to (df_notas, None)
res = minimize(funcion_objetivo, [0.05, 0.15, 0.40], args=(df_notas, None), 
               constraints=cons, bounds=bnds)

w_q, w_t, w_f = res.x

print("=== PESOS ÓPTIMOS ENCONTRADOS ===")
print(f"Quiz por Módulo : {w_q:.2%} (Ratio Q/T: {w_q/w_t:.2f})")
print(f"Tarea por Módulo: {w_t:.2%}")
print(f"Examen Final    : {w_f:.2%}")

# Verificación Post-Optimizacion (Oraculo Dinamico)
y_final_dynamic = get_oracle_decisions_vectorized(res.x, df_notas)
acc = ((calcular_nota_strict(res.x, df_notas) >= 2.95).astype(int) == y_final_dynamic).mean()
print(f"Exactitud del Modelo (Dynamic): {acc:.2%}")

# FPR Calculation
tn = ((y_final_dynamic == 0) & (calcular_nota_strict(res.x, df_notas) < 2.95)).sum()
fp = ((y_final_dynamic == 0) & (calcular_nota_strict(res.x, df_notas) >= 2.95)).sum()
fpr = fp / (fp + tn + 1e-9)
print(f"Prob. de ganar sin merecerlo (FPR): {fpr:.2%}")


Optimizando (Dynamic Oracle)...
=== PESOS ÓPTIMOS ENCONTRADOS ===
Quiz por Módulo : 5.00% (Ratio Q/T: 0.30)
Tarea por Módulo: 16.66%
Examen Final    : 35.00%
Exactitud del Modelo (Dynamic): 94.00%
Prob. de ganar sin merecerlo (FPR): 14.37%


In [69]:
# Actualizar Oraculo con Pesos Finales
df_notas['Aprueba_Humano'] = get_oracle_decisions_vectorized(res.x, df_notas)
# 5. INFORME FINAL DETALLADO (Formato Solicitado)
def generar_reporte_detallado(weights, df, U=2.5):
    # Desempaquetar pesos
    if len(weights)==3: w_q,w_t,w_f=weights
    else: w_q=weights[0]; w_t=weights[1]; w_f=weights[6]
    w_mod = w_q + w_t
    eps = 1e-9
    
    # --- 1. Calcular U factors ---
    # Necesitamos las notas de modulo consolidadas para el Umbral
    n_m1 = (df['M1_Q']*w_q + df['M1_T']*w_t) / (w_mod + eps)
    n_m2 = (df['M2_Q']*w_q + df['M2_T']*w_t) / (w_mod + eps)
    n_m3 = (df['M3_Q']*w_q + df['M3_T']*w_t) / (w_mod + eps)
    n_fin = df['Examen_Final']
    
    comps = np.stack([n_m1, n_m2, n_m3, n_fin], axis=1)
    ratios = np.clip(comps / U, 0, 1)
    U_factors = np.zeros_like(comps)
    for i in range(4):
        others = [j for j in range(4) if j != i]
        U_factors[:, i] = np.prod(ratios[:, others], axis=1)
        
    # --- 2. Construir DataFrame con columnas especificas ---
    rep = df.copy()
    
    # Factores U
    u_m1, u_m2, u_m3, u_fin = U_factors[:, 0], U_factors[:, 1], U_factors[:, 2], U_factors[:, 3]
    
    # Modulo 1
    rep['M1_UQ'] = rep['M1_Q'] * u_m1
    rep['M1_UT'] = rep['M1_T'] * u_m1
    rep['M1'] = n_m1 # Nota Modulo Original
    
    # Modulo 2
    rep['M2_UQ'] = rep['M2_Q'] * u_m2
    rep['M2_UT'] = rep['M2_T'] * u_m2
    rep['M2'] = n_m2
    
    # Modulo 3
    rep['M3_UQ'] = rep['M3_Q'] * u_m3
    rep['M3_UT'] = rep['M3_T'] * u_m3
    rep['M3'] = n_m3
    
    # Examen
    rep['Examen_U'] = rep['Examen_Final'] * u_fin
    
    # --- 3. Notas Finales ---
    # Calculada SIN umbral (Promedio ponderado simple con los pesos dados)
    # Importante: Usamos los pesos globales optimizados (w_q, w_t c/u)
    # Nota: w_q se aplica a M1_Q, M2_Q, M3_Q. 
    # Formula estándar: Sum( Item * peso_item )
    rep['Calculada sin umbral'] = (
        (rep['M1_Q'] + rep['M2_Q'] + rep['M3_Q']) * w_q +
        (rep['M1_T'] + rep['M2_T'] + rep['M3_T']) * w_t +
        rep['Examen_Final'] * w_f
    )
    
    # Calculada CON umbral
    rep['Calculada con umbral'] = (
        (rep['M1_UQ'] + rep['M2_UQ'] + rep['M3_UQ']) * w_q +
        (rep['M1_UT'] + rep['M2_UT'] + rep['M3_UT']) * w_t +
        rep['Examen_U'] * w_f
    )
    
    # Apruebas
    # Grade Drop Floor (Safety Net)
    mask_low = rep['Calculada sin umbral'] < 3.0
    floor_val = 0.8 * rep['Calculada sin umbral']
    rep['Calculada con umbral'] = np.where(
        mask_low & (rep['Calculada con umbral'] < floor_val),
        floor_val,
        rep['Calculada con umbral']
    )

    rep['Aprueba_promedio'] = (rep['Calculada sin umbral'] >= 2.95).astype(int)
    # Aprueba_Humano ya viene en df
    
    return rep

df_final = generar_reporte_detallado(res.x, df_notas)

# Definir Estado para colorear (basado en Umbral vs Humano)
model_pass = (df_final['Calculada con umbral'] >= 2.95).astype(int)
conditions = [
    (df_final['Aprueba_Humano'] == 1) & (model_pass == 1),
    (df_final['Aprueba_Humano'] == 0) & (model_pass == 0),
    (df_final['Aprueba_Humano'] == 1) & (model_pass == 0),
    (df_final['Aprueba_Humano'] == 0) & (model_pass == 1)
]
choices = ['COINCIDE (Pasa)', 'COINCIDE (Pierde)', 'FALSO NEGATIVO', 'FALSO POSITIVO']
df_final['Estado'] = np.select(conditions, choices, default='UNKNOWN')

# Orden de Columnas Solicitado
cols = [
    'M1_Q', 'M1_UQ', 'M1_T', 'M1_UT', 'M1',
    'M2_Q', 'M2_UQ', 'M2_T', 'M2_UT', 'M2',
    'M3_Q', 'M3_UQ', 'M3_T', 'M3_UT', 'M3',
    'Examen_Final', 'Examen_U',
    'Calculada sin umbral', 'Calculada con umbral',
    'Aprueba_Humano', 'Aprueba_promedio', 'Estado'
]

# Formato y Muestra
def color_rows(s):
    if 'FALSO' in str(s['Estado']):
        return ['background-color: #ffcccc'] * len(s)
    return [''] * len(s)

print("=== REPORTE FINAL ESTILO SOLICITADO ===")
# Filter for relevant range [2.5, 3.5]
mask_range = df_final['Calculada sin umbral'].between(2.5, 3.5)
df_focus = df_final[mask_range]

print(f"Mostrando ejemplos en rango [2.5, 3.5] (Total: {len(df_focus)} estudiantes)")

# Sample from focused group
nsample = 20
sample_idx = pd.concat([
    df_focus[df_focus['Estado'].str.contains('FALSO')].sample(min(nsample, len(df_focus[df_focus['Estado'].str.contains('FALSO')]))),
    df_focus[df_focus['Estado'].str.contains('COINCIDE')].sample(min(nsample, len(df_focus[df_focus['Estado'].str.contains('COINCIDE')])))
]).index

format_dict = {c: "{:.2f}" for c in cols if c not in ['Aprueba_Humano', 'Aprueba_promedio', 'Estado']}

display(df_final.loc[sample_idx, cols].sort_values('Estado').style.apply(color_rows, axis=1).format(format_dict))

acc = (model_pass == df_final['Aprueba_Humano']).mean()
print(f"Exactitud Final: {acc:.2%}")

# FPR Calculation
fp = ((model_pass == 1) & (df_final['Aprueba_Humano'] == 0)).sum()
real_neg = (df_final['Aprueba_Humano'] == 0).sum()
fpr = fp / real_neg if real_neg > 0 else 0.0
print(f"Prob. de ganar sin merecerlo (FPR): {fpr:.2%} (FP={fp}/{real_neg})")

=== REPORTE FINAL ESTILO SOLICITADO ===
Mostrando ejemplos en rango [2.5, 3.5] (Total: 162 estudiantes)


,M1_Q,M1_UQ,M1_T,M1_UT,M1,M2_Q,M2_UQ,M2_T,M2_UT,M2,M3_Q,M3_UQ,M3_T,M3_UT,M3,Examen_Final,Examen_U,Calculada sin umbral,Calculada con umbral,Aprueba_Humano,Aprueba_promedio,Estado
688,4.05,4.05,5.00,5.00,4.78,2.56,2.56,2.74,2.74,2.70,4.24,4.24,4.01,4.01,4.06,2.71,2.71,3.45,3.45,1,1,COINCIDE (Pasa)
1878,3.20,3.20,3.14,3.14,3.15,3.73,3.73,3.93,3.93,3.88,3.75,3.75,3.38,3.38,3.47,3.03,3.03,3.34,3.34,1,1,COINCIDE (Pasa)
782,4.76,1.91,5.00,2.00,4.94,0.41,0.41,1.18,1.18,1.00,3.46,1.39,4.39,1.76,4.18,3.30,1.32,3.35,1.47,0,1,COINCIDE (Pierde)
708,3.98,2.88,4.16,3.01,4.12,1.20,0.87,3.13,2.27,2.68,5.00,3.62,5.00,3.62,5.00,1.81,1.81,3.19,2.48,0,1,COINCIDE (Pierde)
795,5.00,2.24,4.29,1.92,4.45,0.85,0.85,1.20,1.20,1.12,3.83,1.71,3.96,1.77,3.93,2.50,1.12,2.93,2.35,0,0,COINCIDE (Pierde)
766,5.00,2.44,5.00,2.44,5.00,0.00,0.00,1.69,1.59,1.30,3.87,1.89,3.47,1.70,3.56,2.35,1.22,2.96,2.37,0,1,COINCIDE (Pierde)
1843,4.36,3.12,4.13,2.96,4.18,3.41,2.44,3.89,2.79,3.78,5.00,3.58,4.09,2.93,4.30,1.79,1.79,3.28,2.53,0,1,COINCIDE (Pierde)
693,5.00,2.94,5.00,2.94,5.00,1.38,1.29,1.63,1.53,1.57,5.00,2.94,4.86,2.86,4.89,2.34,1.47,3.30,2.10,0,1,COINCIDE (Pierde)
607,3.85,0.59,4.83,0.74,4.60,0.12,0.07,0.77,0.47,0.62,4.33,0.66,5.00,0.76,4.85,1.54,0.38,2.72,2.18,0,0,COINCIDE (Pierde)
721,4.02,1.53,3.77,1.43,3.83,0.21,0.19,1.29,1.18,1.04,5.00,1.90,4.21,1.60,4.39,2.28,0.95,2.80,2.24,0,0,COINCIDE (Pierde)


Exactitud Final: 96.10%
Prob. de ganar sin merecerlo (FPR): 9.34% (FP=78/835)


In [70]:
# 6. CÁLCULO DETALLADO (Ejemplos Paso a Paso)
def mostrar_detalle_estudiante(idx, df, weights, U=2.5):
    row = df.loc[idx]
    w_q, w_t, w_f = weights
    w_mod = w_q + w_t
    eps = 1e-9
    
    print(f"\n{'='*60}")
    print(f"ESTUDIANTE ID: {idx} | Estado Humano: {'APRUEBA' if row['Aprueba_Humano'] else 'REPRUEBA'}")
    print(f"{'='*60}")
    
    # 1. Notas de Módulo
    n_m1 = (row['M1_Q']*w_q + row['M1_T']*w_t) / (w_mod + eps)
    n_m2 = (row['M2_Q']*w_q + row['M2_T']*w_t) / (w_mod + eps)
    n_m3 = (row['M3_Q']*w_q + row['M3_T']*w_t) / (w_mod + eps)
    n_fin = row['Examen_Final']
    
    comps = [n_m1, n_m2, n_m3, n_fin]
    names = ['M1', 'M2', 'M3', 'Final']
    
    print("\n1. NOTAS CONSOLIDADAS POR MÓDULO:")
    print(f"   Pesos Internos -> Quiz: {w_q:.2%}, Tarea: {w_t:.2%}")
    for n, val in zip(names, comps):
        print(f"   {n}: {val:.2f}")

    # 2. Ratios y Factores U
    print(f"\n2. FACTORES DE UMBRAL (U={U}):")
    ratios = []
    for val in comps:
        rat = min(1.0, val/U)
        ratios.append(rat)
    
    U_factors = []
    for i in range(4):
        prod = 1.0
        details = []
        for j in range(4):
            if i != j:
                prod *= ratios[j]
                details.append(f"{ratios[j]:.2f}")
        U_factors.append(prod)
        print(f"   U_{names[i]} = {' * '.join(details)} = {prod:.4f}")

    # 3. Contribución Final
    print(f"\n3. CÁLCULO NOTA FINAL:")
    print("   Nota = Sum( Nota_i * Peso_Global_i * U_i )")
    weights_global = [w_mod, w_mod, w_mod, w_f]
    
    total = 0
    for i in range(4):
        term = comps[i] * weights_global[i] * U_factors[i]
        total += term
        print(f"   {names[i]}: {comps[i]:.2f} * {weights_global[i]:.2%} * {U_factors[i]:.4f} = {term:.4f}")
    
    print(f"{'-'*40}")
    
    # STRICT FAILURE RULE CHECK
    raw_val = sum(c*w for c,w in zip(comps, weights_global))
    
    if raw_val < 3.0:
        print(f"   [NOTA REPROBATORIA] Nota Base ({raw_val:.4f}) < 3.0")
        print(f"   -> Aplicando Regla del 80%: Nota Final = 0.8 * Nota Base")
        total = 0.8 * raw_val
    
    print(f"   NOTA FINAL CON UMBRAL: {total:.4f}")
    print(f"   (Calculada sin umbral sería: {raw_val:.4f})")

In [71]:
# 7. VISUALIZACIÓN DE EJEMPLOS
# Buscar un estudiante que caiga en la regla estricta (< 3.0)
print("Buscando ejemplos...")

# Recalcular notas raw para encontrar candidatos
w_q, w_t, w_f = res.x
w_mod = w_q + w_t
local_weights = np.array([w_mod, w_mod, w_mod, w_f])
eps = 1e-9

# Calculo simplificado para busqueda
n_m1 = (df_notas['M1_Q']*w_q + df_notas['M1_T']*w_t) / (w_mod + eps)
n_m2 = (df_notas['M2_Q']*w_q + df_notas['M2_T']*w_t) / (w_mod + eps)
n_m3 = (df_notas['M3_Q']*w_q + df_notas['M3_T']*w_t) / (w_mod + eps)
n_fin = df_notas['Examen_Final']
comps = np.stack([n_m1, n_m2, n_m3, n_fin], axis=1)
raw_grades = np.sum(comps * local_weights, axis=1)

# Candidato Reprobado (Regla 80%)
fail_candidates = df_notas.index[raw_grades < 2.9].tolist()
idx_fail = fail_candidates[0] if fail_candidates else df_notas.index[0]

# Candidato Aprobado
pass_candidates = df_notas.index[raw_grades > 3.5].tolist()
idx_pass = pass_candidates[0] if pass_candidates else df_notas.index[1]

print(f"Ejemplo Reprobado (ID {idx_fail}) vs Aprobado (ID {idx_pass})")

# Mostrar Detalles
mostrar_detalle_estudiante(idx_pass, df_notas, res.x)
mostrar_detalle_estudiante(idx_fail, df_notas, res.x)


Buscando ejemplos...
Ejemplo Reprobado (ID 0) vs Aprobado (ID 606)

ESTUDIANTE ID: 606 | Estado Humano: REPRUEBA

1. NOTAS CONSOLIDADAS POR MÓDULO:
   Pesos Internos -> Quiz: 5.00%, Tarea: 16.66%
   M1: 3.55
   M2: 1.45
   M3: 3.98
   Final: 4.51

2. FACTORES DE UMBRAL (U=2.5):
   U_M1 = 0.58 * 1.00 * 1.00 = 0.5818
   U_M2 = 1.00 * 1.00 * 1.00 = 1.0000
   U_M3 = 1.00 * 0.58 * 1.00 = 0.5818
   U_Final = 1.00 * 0.58 * 1.00 = 0.5818

3. CÁLCULO NOTA FINAL:
   Nota = Sum( Nota_i * Peso_Global_i * U_i )
   M1: 3.55 * 21.67% * 0.5818 = 0.4479
   M2: 1.45 * 21.67% * 1.0000 = 0.3152
   M3: 3.98 * 21.67% * 0.5818 = 0.5016
   Final: 4.51 * 35.00% * 0.5818 = 0.9184
----------------------------------------
   NOTA FINAL CON UMBRAL: 2.1832
   (Calculada sin umbral sería: 3.5257)

ESTUDIANTE ID: 0 | Estado Humano: REPRUEBA

1. NOTAS CONSOLIDADAS POR MÓDULO:
   Pesos Internos -> Quiz: 5.00%, Tarea: 16.66%
   M1: 1.81
   M2: 1.62
   M3: 1.45
   Final: 1.05

2. FACTORES DE UMBRAL (U=2.5):
   U_M1 = 0.6